In [1]:
from utils import process_csv
input_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/MonitorSnifferRx.csv'
output_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'
log_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/log1106_uniform_20#%20_seed_12345.txt'
process_csv(input_file, output_file, log_file)

{47, 39}


In [2]:
from utils import load_data
import numpy as np
import models 

In [3]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [4]:
# np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)


D:\zhuomian\file\弱网中鲁棒性的恶意节点检测\exp\utils.py:443: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:653.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [5]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(processed_data)
print(adj[12])
print(T[12])

tensor(indices=tensor([[0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4],
                       [0, 1, 0, 1, 2, 3, 2, 3, 4, 3, 4]]),
       values=tensor([0.5000, 0.5000, 0.3333, 0.6667, 0.5000, 0.5000, 0.2500,
                      0.5000, 0.2500, 0.5000, 0.5000]),
       size=(5, 5), nnz=11, layout=torch.sparse_coo)
tensor(indices=tensor([[0, 0, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 4, 4],
                       [0, 1, 0, 1, 2, 3, 4, 3, 4, 5, 6, 7, 6, 7]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
       size=(5, 8), nnz=14, layout=torch.sparse_coo)


In [6]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[-1].edge_index.numpy().size)

In [7]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
# print(processed_data[-1].x[:,0:8].numpy().size)
# print(processed_data[0].y.numpy())
print(processed_data[12].edge_index)
print(processed_data[12].edge_weights)
print(processed_data[12].x)

tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 1, 2, 3, 3, 3, 4]])
tensor([1.0000, 0.5000, 0.0000, 0.0000, 0.0000])
tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   3.0000,   0.0000,   0.0000,
           0.0000, 115.5520,  22.0000, -93.5521],
        [ 51.0000,   0.0000,   0.0000,   0.0000,   3.0000,   8.0000,   0.0000,
           0.0000, 115.4168,  22.0000, -93.4169],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          50.0000, 115.5520,  22.0000, -93.5521],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           1.0000,  16.9696, -68.2015, -85.1711],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           1.0000,  25.3437, -68.2015, -93.5452],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           1.0000,  16.9696, 

In [8]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(8, 32)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=2, bias=True)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
)


In [14]:
import time
import torch.optim as optim
from utils import accuracy, auc, precision, recall, f1_score
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
def train(epoch):
    t = time.time()
    train_model.train()
    
    
    optimizer.zero_grad()
    # print(processed_data[0].x[0:8].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(group_num):
        print(i)
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        # print(output)
        labels = processed_data[i].y
        loss_train = criteria(output, labels)
        # print(output[:, 1])
        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()
        
        train_model.eval()
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        # print(output)
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        auc_val = auc(output, labels)
        per_val = precision(output, labels)
        recall_val = recall(output, labels)
        f1_val = f1_score(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
            'auc_val: {:.4f}'.format(auc_val.item())
            'per_val: {:.4f}'.format(per_val),
            'recall_val: {:.4f}'.format(recall_val),
            'f1_val: {:.4f}'.format(f1_val),
              'time: {:.4f}s'.format(time.time() - t))
    return loss_val.item()

SyntaxError: invalid syntax (2614412611.py, line 48)

In [10]:
def test():
    train_model.eval()
    output = train_model(processed_data[group_num-1].x[:, 0:8], processed_data[group_num-1].edge_index, 
                             processed_data[group_num-1].edge_weights, processed_data[group_num-1].x[:, 8:11], adj[group_num-1], T[group_num-1])
    labels = processed_data[group_num-1].y
    
    loss_test = criteria(output, labels)
    acc_test = acc_measure(output, labels)
    auc_test = auc(output, labels)
    per_test = precision(output, labels)
    recall_test = recall(output, labels)
    f1_test = f1_score(output, labels)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
         'auc_test: '.format(auc_test),
            'per_test: {:.4f}'.format(per_test),
            'recall_test: {:.4f}'.format(recall_test),
            'f1_test: {:.4f}'.format(f1_test))
    return acc_test.item()

In [11]:
# torch.autograd.set_detect_anomaly(True)
num_epochs = 1000
early_stopping = 30
val_watch = []
t_total = time.time()
for epoch in range(num_epochs):
    val = train(epoch)  # 训练并获得结果
    if val is None:  # 检查是否返回了None值
        print(f"Warning: train(epoch) returned None for epoch {epoch}. Skipping this iteration.")
        continue
    
    val_watch.append(val)  # 将结果添加到val_watch
    test()
    if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
        print("Early stopping...")
        break
    
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

0
0.6133333333333334
loss_train: 0.6605 acc_train: 0.8000 loss_val: 2.1799 acc_val: 0.4600 auc_val:  per_val: 0.1333 recall_val: 0.8000 f1_val: 0.2286 time: 0.0238s
1
0.25333333333333335
loss_train: 8.7622 acc_train: 0.1000 loss_val: 2.6865 acc_val: 0.2000 auc_val:  per_val: 0.0930 recall_val: 0.8000 f1_val: 0.1667 time: 0.0380s
2
0.8044444444444444
loss_train: 1.4851 acc_train: 0.5000 loss_val: 0.3701 acc_val: 0.8600 auc_val:  per_val: 0.2500 recall_val: 0.2000 f1_val: 0.2222 time: 0.0512s
3
0.4
loss_train: 1.0173 acc_train: 0.8800 loss_val: 1.7010 acc_val: 0.9000 auc_val:  per_val: 0.0000 recall_val: 0.0000 f1_val: 0.0000 time: 0.0657s
4
0.5066666666666667
loss_train: 1.1102 acc_train: 0.9000 loss_val: 1.6014 acc_val: 0.9000 auc_val:  per_val: 0.0000 recall_val: 0.0000 f1_val: 0.0000 time: 0.0819s
5
0.45333333333333337
loss_train: 1.6622 acc_train: 0.9000 loss_val: 2.3099 acc_val: 0.9000 auc_val:  per_val: 0.0000 recall_val: 0.0000 f1_val: 0.0000 time: 0.0974s
6
0.6577777777777778
lo